<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Experimento 4 AutoGluon</h4>
</center>

In [2]:
# !pip install autogluon
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [4]:
df = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_ventas_product.csv", header = None)
df.columns = ["timestamp", "item_id", "target"]
df.head(5)

,timestamp,item_id,target
0,2017-01-01,20001,934.77222
1,2017-01-01,20002,550.15707
2,2017-01-01,20003,1063.45835
3,2017-01-01,20004,555.91614
4,2017-01-01,20005,494.27011


In [5]:
df_product = pd.read_csv("C:/Users/usuario/otros/Desktop/MCD/6 Labo III/Forecasting Problem/Datasets/df_products.csv", header = None)
df_product.columns = ["item_id", "cat1", "cat2", "cat3", "brand", "sku_size"]
df_product.head(5)

,item_id,cat1,cat2,cat3,brand,sku_size
0,20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900
1,20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
2,20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
3,20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
4,20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900


In [6]:
data = TimeSeriesDataFrame.from_data_frame(
    df,
    id_column="item_id",
    timestamp_column="timestamp",
    static_features_df=df_product,
)
data.head()

,,target
item_id,timestamp,
20001,2017-01-01,934.77222
20002,2017-01-01,550.15707
20003,2017-01-01,1063.45835
20004,2017-01-01,555.91614
20005,2017-01-01,494.27011


In [7]:
data.static_features.head()

,cat1,cat2,cat3,brand,sku_size
item_id,,,,,
20280,HC,ROPA LAVADO,Liquido,LIMPIEX,900
20180,HC,ROPA LAVADO,Liquido,LIMPIEX,450
20332,HC,ROPA LAVADO,Liquido,LIMPIEX,120
20222,HC,ROPA LAVADO,Liquido,LIMPIEX,450
20288,HC,ROPA LAVADO,Liquido,LIMPIEX,900


In [8]:
predictor = TimeSeriesPredictor(prediction_length=2, freq="M").fit(data)
predictions = predictor.predict(data)

Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to 'AutogluonModels\ag-20240620_184733'
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.11.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          8
GPU Count:          0
Memory Avail:       0.89 GB / 7.88 GB (11.2%)
Disk Space Avail:   1.50 GB / 110.21 GB (1.4%)
	We recommend a minimum available disk space of 10 GB, and large datasets may require more.

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequency 'None' has been resa

In [9]:
predictions.head(5)

mean         0.1          0.2          0.3  \
item_id timestamp                                                       
20001   2020-01-31  1308.582889  981.334997  1109.385721  1158.895327   
        2020-02-29  1340.885983  996.575696  1120.081666  1176.752878   
20002   2020-01-31  1022.033347  688.065266   806.044768   862.203138   
        2020-02-29  1036.703843  696.024846   819.117393   866.520560   
20003   2020-01-31   775.661168  535.438412   632.413816   661.029530   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
20001   2020-01-31  1249.709015  1310.452945  1347.684600  1403.106141   
        2020-02-29  1263.272352  1338.106872  1384.376967  1441.920529   
20002   2020-01-31   949.717655  1017.997282  1053.126001  1111.148079   
        2020-02-29   955.855598  1037.030730  1080.271629  1136.383057   
20003   2020-01-31   724.849580   777.758985   808.522988   844.481995   

                            0.8          0.9  
item_id timestamp                             
20001   2020-01-31  1484.516026  1593.196760  
        2020-02-29  1533.349055  1643.228595  
20002   2020-01-31  1196.638750  1305.159017  
        2020-02-29  1220.901343  1332.603306  
20003   2020-01-31   899.937066   961.421672

In [10]:
predictions_v1 = predictions.copy()
predictions_v1 = predictions_v1.reset_index()
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]
# predictions_v1 = predictions_v1[["item_id", "timestamp", "0.4"]]
predictions_v1 = predictions_v1[predictions_v1.timestamp == "2020-02-29"]
predictions_v1 = predictions_v1.drop(columns = {"timestamp"})
predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "mean":"tn"})
# predictions_v1 = predictions_v1.rename(columns = {"item_id":"product_id", "0.4":"tn"})
predictions_v1.head(5)

,product_id,tn
1,20001,1340.885983
3,20002,1036.703843
5,20003,788.005687
7,20004,601.573492
9,20005,580.468542


In [11]:
# predictions_v1.to_csv("prediccion_autogluon_static_features_wql_04.csv", sep = ",", index = False)
predictions_v1.to_csv("C:/Users/usuario/otros/Desktop\MCD/6 Labo III/Forecasting Problem/Outputs/prediccion_autogluon_static_features_wql.csv", sep = ",", index = False)